In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
from tqdm import tqdm
#import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [360]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATIX[c1.Index, c2.Index] = DIST_MATIX[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

NUM_CITIES = len(CITIES)
CITIES.head()


,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


In [361]:
# def fitness(solution):
#     #return np.sum(np.linalg.norm(cities.iloc[solution].diff().values, axis=1))
#     sum_distance = 0
#     for i in range(NUM_CITIES):
#         sum_distance += dist_matrix[solution[i], solution[(i+1)%NUM_CITIES]]
#         ic(sum_distance)

In [362]:
# init_solution = np.random.permutation(NUM_CITIES)  
# init_solution

# city = 2
# print(cities_names[city])
# closest_city = np.argmin(dist_matrix[city])
# print(closest_city, cities_names[closest_city])
# None

In [ ]:
from dataclasses import dataclass


@dataclass
class Individual:
    path: np.ndarray
    cost: float = None

def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATIX[c1, c2]
    return tot_cost

#sum distances, but does't need the last element of the vector to be equal to the first
def tsp_cost_cutTail(tsp: Individual):
    tot_cost = 0
    for i in range(NUM_CITIES):
        tot_cost += DIST_MATIX[tsp.path[i], tsp.path[(i+1)%NUM_CITIES]]
    return tot_cost

def inversion_mutation(tsp: Individual):
    new_tsp = Individual(tsp.path, tsp.cost)
    i, j = np.random.choice(len(new_tsp.path), 2, replace=False)
    if i > j:
        i, j = j, i
    new_tsp.path[i:j] = new_tsp.path[i:j][::-1]
    new_tsp.cost = tsp_cost_cutTail(new_tsp)
    return new_tsp


def inver_over_crossover(parent1: Individual, parent2: Individual):
    new_path = np.atleast_1d(np.array(parent1.path.copy()))
    parent2_path = np.atleast_1d(np.array(parent2.path))

    length = len(new_path)
    
    i = np.random.randint(0, length)
    first_city = new_path[i]
    
    index_in_parent2 = np.where(parent2_path == first_city)[0][0]
    j = (index_in_parent2 + 1) % length  
    second_city = parent2_path[j]
    
    i = np.where(new_path == first_city)[0][0]
    j = np.where(new_path == second_city)[0][0]
    
    if i > j:
        i, j = j, i
    
    # inversion
    new_path[i:j+1] = new_path[i:j+1][::-1]
    
    return Individual(new_path, tsp_cost_cutTail(Individual(new_path)))



In [364]:
visited = np.full(len(CITIES), False)
dist = DIST_MATIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])

tot_cost = 0
for c1, c2 in zip(tsp, tsp[1:]):
    tot_cost += DIST_MATIX[c1, c2]
logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

DEBUG:root:step: Acheng -> Harbin (33.60km)
DEBUG:root:step: Harbin -> Shuangcheng (53.02km)
DEBUG:root:step: Shuangcheng -> Yushu (61.85km)
DEBUG:root:step: Yushu -> Wuchang (47.68km)
DEBUG:root:step: Wuchang -> Shulan (59.07km)
DEBUG:root:step: Shulan -> Jishu (17.91km)
DEBUG:root:step: Jishu -> Jilin city (50.81km)
DEBUG:root:step: Jilin city -> Jiutai (65.06km)
DEBUG:root:step: Jiutai -> Dehui (43.68km)
DEBUG:root:step: Dehui -> Changchun (78.49km)
DEBUG:root:step: Changchun -> Gongzhuling (59.12km)
DEBUG:root:step: Gongzhuling -> Siping (54.24km)
DEBUG:root:step: Siping -> Liaoyuan (71.76km)
DEBUG:root:step: Liaoyuan -> Meihekou (60.38km)
DEBUG:root:step: Meihekou -> Panshi (55.16km)
DEBUG:root:step: Panshi -> Huadian (56.40km)
DEBUG:root:step: Huadian -> Jiaohe (96.49km)
DEBUG:root:step: Jiaohe -> Dunhua (82.15km)
DEBUG:root:step: Dunhua -> Helong (110.22km)
DEBUG:root:step: Helong -> Longjing (42.88km)
DEBUG:root:step: Longjing -> Yanji (14.70km)
DEBUG:root:step: Yanji -> Tumen 

In [ ]:
# best_solution = tsp[0:NUM_CITIES]
# best_solution_cost = tsp_cost_cutTail(best_solution)

SIZE_POPULATION = 40

# Initialize best individual
best_individual = Individual([], np.inf)

# Random population
population = []
for j in range(SIZE_POPULATION - 1):
    new_path = np.random.choice(range(NUM_CITIES), size=NUM_CITIES, replace=False)
    new_path_cost = tsp_cost_cutTail(Individual(new_path, 0))
    population.append(Individual(new_path, new_path_cost))

# Add best path found with fast algorithm
population.append(Individual(path=tsp[0:NUM_CITIES], cost=tsp_cost_cutTail(Individual(tsp[0:NUM_CITIES]))))

population.sort(key=lambda x: x.cost)

ic(population[0])






ic| population[0]: Individual(path=[0,
                                    186,
                                    505,
                                    677,
                                    570,
                                    508,
                                    285,
                                    264,
                                    289,
                                    93,
                                    46,
                                    157,
                                    513,
                                    328,
                                    384,
                                    419,
                                    212,
                                    252,
                                    114,
                                    197,
                                    359,
                                    636,
                                    550,
                                    229,
                    

[0,
 186,
 505,
 677,
 570,
 508,
 285,
 264,
 289,
 93,
 46,
 157,
 513,
 328,
 384,
 419,
 212,
 252,
 114,
 197,
 359,
 636,
 550,
 229,
 414,
 173,
 396,
 397,
 198,
 290,
 101,
 451,
 395,
 233,
 23,
 507,
 240,
 191,
 399,
 644,
 539,
 516,
 174,
 28,
 573,
 412,
 450,
 232,
 358,
 681,
 629,
 172,
 235,
 685,
 382,
 154,
 239,
 192,
 457,
 469,
 356,
 207,
 5,
 696,
 699,
 514,
 435,
 434,
 532,
 16,
 554,
 545,
 506,
 538,
 540,
 138,
 108,
 671,
 492,
 519,
 95,
 327,
 12,
 287,
 329,
 616,
 33,
 139,
 34,
 59,
 283,
 338,
 418,
 650,
 87,
 142,
 557,
 430,
 79,
 711,
 105,
 81,
 126,
 36,
 543,
 18,
 343,
 481,
 133,
 544,
 234,
 605,
 444,
 346,
 531,
 183,
 530,
 143,
 535,
 309,
 218,
 31,
 549,
 637,
 470,
 509,
 52,
 388,
 123,
 716,
 146,
 27,
 459,
 194,
 41,
 44,
 220,
 318,
 38,
 112,
 196,
 504,
 443,
 645,
 718,
 708,
 690,
 40,
 305,
 617,
 523,
 266,
 326,
 344,
 403,
 292,
 199,
 494,
 613,
 147,
 366,
 497,
 375,
 614,
 103,
 6,
 21,
 100,
 609,
 475,
 182,
 34

In [ ]:

reps = 100_000
prob = 1

for i in tqdm(range(reps)):
    if np.random.rand() < 0.5:
        parent1 = population[np.random.randint(0, len(population) // 2)]
        parent2 = population[np.random.randint(0, len(population) // 2)]
    else:
        parent1 = population[0]
        parent2 = population[1]

    offsprings = []
    
    if i % (reps // 10) == 0:
        prob = max(0.1, prob - 0.1)

    # Inver-over or inversion
    if np.random.rand() < prob:
        for k in range(int(SIZE_POPULATION/5)):
            sol = inver_over_crossover(parent1, parent2)
            offsprings.append(sol)
    else:
        for j in range(int(SIZE_POPULATION/5)):
            sol = inversion_mutation(parent1)
            offsprings.append(sol)

    population.extend(offsprings)
    population = sorted(population, key=lambda x: x.cost)[:10]

    if best_individual.cost > population[0].cost:
        best_individual = population[0]

ic(best_individual)



100%|██████████| 100000/100000 [04:34<00:00, 363.86it/s]
ic| best_individual: Individual(path=array([147, 514, 595, 493, 634, 638, 571, 170, 592, 459, 627, 302, 291,
                            565, 207, 309, 550, 512, 429, 103, 511, 371, 520, 504,  69,  64,
                            597, 441, 496,  99, 492, 559, 423, 178, 290, 443, 288, 482, 174,
                            197, 612,  97, 697, 603, 545, 455, 486, 519, 583, 242, 358, 340,
                            322, 649, 414, 639, 534, 214, 626, 475, 294, 352, 683, 195, 351,
                            517, 337,  35, 270, 182, 440, 293, 537, 551, 644, 608, 175,  23,
                            263, 158, 535, 259, 383, 650, 446, 349, 579, 694, 226, 136, 539,
                            474, 316, 216, 668, 555, 463, 532, 621, 389, 160, 471,  40, 452,
                            586, 284, 376, 477, 480, 399, 688, 645, 570, 470, 289, 620, 700,
                            528, 656, 691,  42, 544, 323, 434, 577, 479, 279, 596, 518, 64

Individual(path=array([147, 514, 595, 493, 634, 638, 571, 170, 592, 459, 627, 302, 291,
       565, 207, 309, 550, 512, 429, 103, 511, 371, 520, 504,  69,  64,
       597, 441, 496,  99, 492, 559, 423, 178, 290, 443, 288, 482, 174,
       197, 612,  97, 697, 603, 545, 455, 486, 519, 583, 242, 358, 340,
       322, 649, 414, 639, 534, 214, 626, 475, 294, 352, 683, 195, 351,
       517, 337,  35, 270, 182, 440, 293, 537, 551, 644, 608, 175,  23,
       263, 158, 535, 259, 383, 650, 446, 349, 579, 694, 226, 136, 539,
       474, 316, 216, 668, 555, 463, 532, 621, 389, 160, 471,  40, 452,
       586, 284, 376, 477, 480, 399, 688, 645, 570, 470, 289, 620, 700,
       528, 656, 691,  42, 544, 323, 434, 577, 479, 279, 596, 518, 643,
       283, 344,  29, 682,  28, 387, 719, 215,  49,  33, 347, 200,  72,
       188, 325, 355, 369, 122, 346,   9, 657, 613,  92, 624, 581, 298,
       239,  84, 665, 686, 619, 248, 488, 238, 542, 397, 320, 687, 404,
       374, 457,  60, 449,   3, 227, 269, 416, 5

In [ ]:

for i in range(len(best_individual.path) - 1):
    city = best_individual.path[i]
    closest = best_individual.path[i + 1]
    
    logging.debug(
        f"Step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} "
        f"({DIST_MATIX[city, closest]:.2f} km)"
    )

# add last edge
city = best_individual.path[-1]
closest = best_individual.path[0]
logging.debug(
    f"Step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} "
    f"({DIST_MATIX[city, closest]:.2f} km)"
)


DEBUG:root:Step: Gaocheng -> Songyuan (1148.59 km)
DEBUG:root:Step: Songyuan -> Xiantao (1917.46 km)
DEBUG:root:Step: Xiantao -> Shenzhen (869.95 km)
DEBUG:root:Step: Shenzhen -> Yangquan (1699.12 km)
DEBUG:root:Step: Yangquan -> Yanshi (355.15 km)
DEBUG:root:Step: Yanshi -> Wuchuan (1485.50 km)
DEBUG:root:Step: Wuchuan -> Haicheng (501.66 km)
DEBUG:root:Step: Haicheng -> Xiangtan (592.25 km)
DEBUG:root:Step: Xiangtan -> Renqiu (1238.85 km)
DEBUG:root:Step: Renqiu -> Xuzhou (501.54 km)
DEBUG:root:Step: Xuzhou -> Kunshan (476.49 km)
DEBUG:root:Step: Kunshan -> Jiyuan (880.80 km)
DEBUG:root:Step: Jiyuan -> Wenchang (1722.43 km)
DEBUG:root:Step: Wenchang -> Honggang (3245.31 km)
DEBUG:root:Step: Honggang -> Langfang (1019.73 km)
DEBUG:root:Step: Langfang -> Tumen (1164.66 km)
DEBUG:root:Step: Tumen -> Simao (3475.76 km)
DEBUG:root:Step: Simao -> Pizhou (2096.70 km)
DEBUG:root:Step: Pizhou -> Dingzhou (536.30 km)
DEBUG:root:Step: Dingzhou -> Sihui (1694.91 km)
DEBUG:root:Step: Sihui -> Luo